In [8]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core pypdf

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
import tempfile
from pypdf import PdfReader
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver
from langchain.document_loaders import PyPDFLoader

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
    tmp_file.write(uploaded_file.read())
    tmp_file_path = tmp_file.name

# Load and split the PDF
loader = PyPDFLoader(tmp_file_path)
pages = loader.load_and_split()

In [ ]:
files = file_uploader("Please select a PDF file.", type="pdf")

In [10]:
graph = Neo4jGraph()

In [12]:
loader = TextLoader(file_path="dum.txt") #this is the text that'll be read
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
documents = text_splitter.split_documents(documents=docs)

In [5]:
llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")
llm_transformer = LLMGraphTransformer(llm=llm)

/tmp/ipykernel_21711/3924532301.py:1: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")


In [22]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [24]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [6]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [7]:
test_docs = TextLoader(file_path="dum.txt").load() #same as above, diff chunk_size lang kasi di kaya ng machine ko

test_documents = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=75).split_documents(documents=test_docs)

In [13]:
#embeddings
#test vector store
db = Neo4jVector.from_documents(
    test_documents, embeddings, url= os.environ["NEO4J_URI"],username=os.environ["NEO4J_USERNAME"], password= os.environ["NEO4J_PASSWORD"]
)


In [10]:
# Now we initialize from existing graph
existing_graph = Neo4jVector.from_existing_graph(
    embedding= embeddings,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)
result = existing_graph.similarity_search("who are the main characters of the story and what are their role")

In [11]:
print(result)

[Document(metadata={'source': 'dum.txt'}, page_content='\ntext: In the town of Rivershade, nestled by the edge of a vast, mist-covered river, lives a multitude of characters, each unaware of how deeply their lives are intertwined until fate conspires to bring them together.'), Document(metadata={'source': 'dum.txt'}, page_content='\ntext: As night falls, the group prepares for a perilous journey across the river’s depths to uncover the secrets that could either save or doom their town.'), Document(metadata={'source': 'dum.txt'}, page_content="\ntext: One evening, Sophia, a journalist from the city, arrives in Rivershade to write an article about the town’s history. She's been following a story about missing people who disappeared along the river, and her investigation leads her to Emmett, the quiet"), Document(metadata={'source': 'dum.txt'}, page_content='\ntext: The next morning, Declan, a young construction worker, is assigned to repair an old bridge that crosses the river. As he ins

In [16]:
query = "What is the nature of the relationship between the river and the people of Rivershade? How does the town's dependence on the river influence its culture, history, and collective psyche?"
test = db.similarity_search_with_score(query, k=10)

print(test)

for doc, score in test:
        print("-" * 80)
        print("Score: ", score)
        print(doc.page_content)
        print("-" * 80)

[(Document(metadata={'source': 'dum.txt'}, page_content='In the town of Rivershade, nestled by the edge of a vast, mist-covered river, lives a multitude of characters, each unaware of how deeply their lives are intertwined until fate conspires to bring them together.'), 0.8958382606506348), (Document(metadata={'source': 'dum.txt'}, page_content='they finally put the pieces together: the river is not just a source of livelihood, but a living, breathing entity that has shaped Rivershade’s history in more ways than they could have imagined. The river’s current was altered years ago—by a deal struck'), 0.8910050392150879), (Document(metadata={'source': 'dum.txt'}, page_content='along the river, and her investigation leads her to Emmett, the quiet son of a wealthy family who has a long-standing connection to the town’s founding. Emmett has always been a bit of an outsider, but he’s recently been returning to Rivershade after'), 0.8790483474731445), (Document(metadata={'source': 'dum.txt'}, 

In [17]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [18]:
#create entities or the subj on your query e.g "who is maria?" possible entity is maria then 'Maria' will be search thru the graph DB
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description=""" Extract all the possible subject in the text""",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization, person or the subject as entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)
llm_trans= OllamaFunctions(model="mistral", temperature=0, format="json")
entity_chain = llm_trans.with_structured_output(Entities)

In [24]:
entity_chain.invoke("who is sophia?")

Entities(names=['sophia'])

In [19]:
# the 'Maria will be used here, on graph retreiver 
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
# print(graph_retriever("What do the symbols on the bridge represent, and how are they connected to the ancient power within the river?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Ne

Declan - LIVES_NEAR -> River
Clara - ASSOCIATED_WITH -> River


In [20]:
def full_retriever(question: str):
    print(question)
    graph_data = graph_retriever(question)
    # docs_with_score = vector_index.similarity_search_with_score(question, k=2)
    # docs_with_score = db.similarity_search_with_score(question, k=2)
    # vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    print(graph_data)
    docs_with_score = db.similarity_search_with_score(question, k=4)
    page_contents = [doc[0].page_content for doc in docs_with_score]
    print(page_contents)

    final_data = f"""Relationships:
{graph_data}
facts:
{page_contents}
    """
    return final_data

In [24]:
# combination of embedding and graph search
template = """
Answer the question based only on the following context:
{context}

You are an advanced AI designed to analyze and synthesize information from a single provided file: {context} containing both factual details and relationship connections about a subject. 
When answering a question, do not simply extract text but instead interpret and expand upon the provided information by logically inferring connections and implications. 
Identify key facts, analyze relationships, and generate well-structured responses that go beyond surface-level details while maintaining accuracy and coherence. 
Use contextual reasoning to provide insightful and relevant answers. If the required information is not found, acknowledge the limitation while avoiding speculation. 
Always maintain a neutral, well-supported, and logically sound tone in your responses.

when answering a question be confident and make it so like you're the one answering the question based from your knowldege not from external source

Question: {question}
Use natural human language 


Answer:"""
llm2 = OllamaFunctions(model="mistral", temperature=0, format="json")
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {   
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm2
    | StrOutputParser()
)

In [25]:
#ask a question
chain.invoke(input="Why was the letter, addressed to Margaret, hidden in a book at the library? What does this suggest about the message’s purpose and the history of the town’s leadership?")

Why was the letter, addressed to Margaret, hidden in a book at the library? What does this suggest about the message’s purpose and the history of the town’s leadership?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Ne

Margaret - GOVERNED_BY -> Rivershade
Eleanor - SHAREDFINDINGS -> Margaret
['It begins with Eleanor, a reclusive librarian, who finds an old, weathered letter in a forgotten book while cleaning the shelves at the Rivershade Public Library. The letter is addressed to Margaret, the town’s mayor, but it is marked with no date or', 'addressed to Margaret, the town’s mayor, but it is marked with no date or sender. Curious, Eleanor reads the contents:', 'Eleanor, fascinated, decides to investigate the mysterious message. Little does she know, Margaret, the mayor, is dealing with her own problems—namely, a growing suspicion about the town’s historic preservation project that her father had started before', 'As Sophia digs deeper into Emmett’s past, she uncovers a startling connection between his great-great-grandfather and the letter Eleanor discovered. The letter, it turns out, was meant for Emmett’s ancestor, a man who was believed to have disappeared']


"The letter, addressed to Margaret, was hidden in a book at the library likely because someone wanted its contents to be discovered. This suggests that the message had important information or implications for Margaret and possibly the town's leadership. The fact that it was found in a forgotten book implies a connection between the letter and the town's history, which might be related to the historic preservation project started by Margaret's father."